# LLM LangChain : Model, Prompt and Output

In [1]:
from dotenv import load_dotenv, find_dotenv
import os
_ = load_dotenv(find_dotenv()) # read local .env file

# openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
import os
import openai

openai.api_base = "http://localhost:12346/v1"
openai.api_key = "EMPTY"

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0.7,  
    )
    return response.choices[0].message["content"]

get_completion("What is 1+1?", model="MPT-7B-Instruct")

'1+1 is 2.'

In [3]:
from langchain.chat_models import ChatOpenAI

# Model Creation

chat = ChatOpenAI(temperature=0.0, model='MPT-7B-Instruct', openai_api_key="EMPTY", openai_api_base="http://localhost:12346/v1")
chat

ChatOpenAI(verbose=False, callbacks=None, callback_manager=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='MPT-7B-Instruct', temperature=0.0, model_kwargs={}, openai_api_key='EMPTY', openai_api_base='http://localhost:12346/v1', openai_organization='', openai_proxy='', request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None)

In [4]:
# Defining Prompt template

from langchain.prompts import ChatPromptTemplate

template_string = """Write the code in {language} for the text which is delimmited \
by triple backticks\
text: ```{text}```
"""

prompt_template = ChatPromptTemplate.from_template(template_string)

print  (prompt_template.messages[0].prompt, prompt_template.input_variables)

# formating message using template
prompt_message = prompt_template.format_messages(language='Python', text='the sum of n numbers' )

print ('prompt_message: ', prompt_message)

# calling LLM 

response = chat(prompt_message)

print('response: ', response)

input_variables=['language', 'text'] output_parser=None partial_variables={} template='Write the code in {language} for the text which is delimmited by triple backtickstext: ```{text}```\n' template_format='f-string' validate_template=True ['language', 'text']
prompt_message:  [HumanMessage(content='Write the code in Python for the text which is delimmited by triple backtickstext: ```the sum of n numbers```\n', additional_kwargs={}, example=False)]
response:  content="```n = int(input('Enter a number: '))\nsum = 0\nfor i in range(n):\n    sum += i\nprint(sum)```" additional_kwargs={} example=False


In [5]:
# Extarcting information from text
from langchain.prompts import ChatPromptTemplate

template_string = """Extract the information from the text which is delimmited \
by triple backticks\

Person: names of the persons otherwise output ""
Meeting Place: the place of meeting otherwise output ""
Day : day of the meeting otherwise output ""
Date: date of the meeting expmple Monday, Tuesday ... otherwise output ""

convert extracted information into JSON format using keys
Person
Meeting Place
Date
Day

text: ```{text}```
"""

prompt_template = ChatPromptTemplate.from_template(template_string)

print  (prompt_template.messages[0].prompt, prompt_template.input_variables)

# formating message using template
prompt_message = prompt_template.format_messages(text='Ram meets Laxman on on 10th January 2013 in Delhi, India' )

print ('prompt_message: ', prompt_message)

# calling LLM 

response = chat(prompt_message)

print (response.content, type(response.content))



input_variables=['text'] output_parser=None partial_variables={} template='Extract the information from the text which is delimmited by triple backticks\nPerson: names of the persons otherwise output ""\nMeeting Place: the place of meeting otherwise output ""\nDay : day of the meeting otherwise output ""\nDate: date of the meeting expmple Monday, Tuesday ... otherwise output ""\n\nconvert extracted information into JSON format using keys\nPerson\nMeeting Place\nDate\nDay\n\ntext: ```{text}```\n' template_format='f-string' validate_template=True ['text']
prompt_message:  [HumanMessage(content='Extract the information from the text which is delimmited by triple backticks\nPerson: names of the persons otherwise output ""\nMeeting Place: the place of meeting otherwise output ""\nDay : day of the meeting otherwise output ""\nDate: date of the meeting expmple Monday, Tuesday ... otherwise output ""\n\nconvert extracted information into JSON format using keys\nPerson\nMeeting Place\nDate\nDay

In [14]:
# output is str convert it into dictionary
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

person_schema = ResponseSchema(name="Persons", description="names of the persons otherwise output None", type='list')
meeting_place_schema = ResponseSchema(name="Meeting Place", description="the place of meeting otherwise output None")
date_schema = ResponseSchema(name="Date", description="date of the meeting  otherwise output None")
day_schema = ResponseSchema(name="Day", description="day of the meeting for example Monday, Tuesday ... otherwise output None")


output_parser = StructuredOutputParser.from_response_schemas([person_schema, meeting_place_schema, date_schema, day_schema])
format_instructions = output_parser.get_format_instructions()

print (format_instructions)

template_string_format = """Extract the information from the text 

Persons: names of the persons otherwise output None
Meeting Place: the place of meeting otherwise output None
Day : day of the meeting for example Monday, Tuesday ... otherwise output None
Date: date of the meeting  otherwise output None

text: {text}
{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=template_string_format)

prompt_message = prompt.format_messages(text='Ram meets Laxman on on 10th January 2013 in Delhi, India', 
                                format_instructions=format_instructions)

print(prompt_message[0].content)

response = chat(prompt_message)

print(response.content)

print (output_parser.parse(response.content), type(output_parser.parse(response.content)))

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"Persons": list  // names of the persons otherwise output None
	"Meeting Place": string  // the place of meeting otherwise output None
	"Date": string  // date of the meeting  otherwise output None
	"Day": string  // day of the meeting for example Monday, Tuesday ... otherwise output None
}
```
Extract the information from the text 

Persons: names of the persons otherwise output None
Meeting Place: the place of meeting otherwise output None
Day : day of the meeting for example Monday, Tuesday ... otherwise output None
Date: date of the meeting  otherwise output None

text: Ram meets Laxman on on 10th January 2013 in Delhi, India
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"Persons": list  // names of the persons otherwise output None
	"M